In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.models as models
from torchvision import transforms
import seaborn as sns
import json
import cv2
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils import data as data_torch
from PIL import Image
from sentence_transformers import SentenceTransformer

In [4]:
image_ex_path="/home/jupyter/Hateful_Memes_Data/babyData/01235.png"
data_ex_path="/home/jupyter/Hateful_Memes_Data/babyData/train.jsonl"
bert_path="/home/jupyter/ThreeDAnime/bert_folder"

In [5]:
data = []
with open(data_ex_path) as f:
    for el in f:
        data.append(json.loads(el))

In [6]:
data[0]

{'id': 42953,
 'img': 'img/42953.png',
 'label': 0,
 'text': 'its their character not their color that matters'}

In [150]:
class SimpleModel(nn.Module):
    '''
    A really simple network architecture
    '''
    def __init__(self,input_size, output_size, n_layers,  layer_size, skip_connection=False):
        super(SimpleModel,self).__init__()
        
        self.input_lay=nn.Linear(input_size,layer_size)
        self.intermediate_layers:[nn.Sequential]=[]
        for n in range(n_layers):
            self.intermediate_layers.append(nn.Sequential((nn.Linear(layer_size,layer_size)),nn.ELU()))
        
        self.output_lay=nn.Linear(layer_size,output_size)
    
    def forward(self,x):
    
        if len(x)!=2:
           raise Exception("expected batch tuple of image features and text features ")
       
        x=torch.cat((x[0],x[1]),axis=1)
        x=self.input_lay(x)
        x=F.elu(x)
        for n in self.intermediate_layers:
            x=n(x)
        x=self.output_lay(x)
        x=F.softplus(x,threshold=1)
        return x
        

In [52]:
class Dataset_Handle_Simple(data_torch.Dataset):
        def __init__(self, imageData, textData,labels):

            self.imgData=imageData 
            self.textData=textData
            self.labels=labels

        def __len__(self):
            'Denotes the total number of samples'
            return len(self.labels)

        def __getitem__(self, index):
            'Generates one sample of data'
            # Select sample

            return self.imgData[index],self.textData[index],self.labels[index]


In [64]:
  
def train_model_loss(model, epochs,  training_data_handler, learn_rate, trainloader):
    train_loss=[]
    MAX_EPOCHS=epochs
    
    if trainloader == None:
        trainloader=torch.utils.data.DataLoader(training_data_handler, batch_size=2, shuffle=True,
                                                num_workers=0)
    
    distance=nn.L1Loss(reduction="sum") #I AM BEING SO LAZY  THIS HSOULD BE. CROSSENTROPY, BUT MEHHHHHH
    
    optimzer=torch.optim.Adam(model.parameters(),lr=learn_rate)

    cycle=(iter(trainloader))
    temp_train_losses = []
    
    for epoch in range(MAX_EPOCHS):
        cycle=(iter(trainloader))
        temp_train_losses_batch = []
        for i in range(len(cycle)):
            relevant_data=next(cycle)
           
            input_img=relevant_data[0]
            input_txt=relevant_data[1]
            ground_truth=relevant_data[2]

            output=model((input_img.float(),input_txt.float()))

            loss=distance(output,ground_truth)
            temp_train_losses_batch.append(loss.detach().numpy())
            optimzer.zero_grad()
            
            if epoch !=0:
           

                loss.backward()
                optimzer.step()
         
        temp_train_losses.append(np.array(temp_train_losses_batch).mean())

    return temp_train_losses

In [82]:
#https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a THE ANIMAL THIS IS MOST ASSoICATED with = absolutely need to swap out with mask rcnn
def extract_image_features_stupid(data:[], model:torchvision.models.resnet.ResNet, size:(),  max:int=10)->torch.Tensor:
    '''
    THIS NEEDS TO BE DONE IN BATCHES OR YOU'LL RUN OUT OF MEMORY
    Converts. resnet features about. animals for use in our meme model 
    '''
    transform = transforms.Compose(
    [transforms.Resize(size),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    image_transformed=torch.empty(len(data),3,size[0],size[1])
    
    for image_counter in range(len(data)):  
        img_mod=transform(data[image_counter]) 
        image_transformed[image_counter]=img_mod
        image_transformed[image_counter]
    
    return model(image_transformed).detach()
    
    

In [83]:
def extract_text_features_simple(data:[str], model:SentenceTransformer):
    return model.encode(data)
    
    

In [35]:
model_text=SentenceTransformer(bert_path)

In [18]:
im=Image.open(image_ex_path)
inception = models.resnet18()

In [40]:
#THIS NEEDS TO BE DONE IN BATCHES 
imageData=extract_image_features_stupid([im,im],inception,(299,299))

In [145]:
textData=extract_text_features_simple([data[0]["text"],data[1]["text"]],model_text)

In [188]:
model_all=SimpleModel(imageData.shape[1]+textData[0].shape[0],1,1,20)

In [195]:
handler=Dataset_Handle_Simple(textData,imageData,torch.tensor([[1],[1]]))

In [196]:
loss=train_model_loss(model_all,30,handler,.01,None)

In [197]:
results=model_all((imageData,torch.tensor(textData)))
print(results)

tensor([[0.8542],
        [0.8542]], grad_fn=<SoftplusBackward>)


In [ ]:
loss

In [137]:
loss

[1.0985398,
 1.0985398,
 0.13905382,
 0.02227515,
 0.0040512085,
 0.00086164474,
 0.00021111965,
 5.865097e-05,
 1.8239021e-05,
 6.198883e-06,
 2.3841858e-06,
 9.536743e-07,
 4.7683716e-07,
 2.3841858e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]